# Vector Wind Analysis with Skyborn Windspharm

This notebook demonstrates the capabilities of the `skyborn.windspharm` module for spherical harmonic vector wind analysis. The module provides tools for computing vorticity, divergence, streamfunction, velocity potential, and performing Helmholtz decomposition of atmospheric wind fields.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skyborn.windspharm import VectorWind
from skyborn.spharm import gaussian_lats_wts

# Set up plotting
plt.style.use('default')
%matplotlib inline

## Creating Sample Wind Data

First, let's create some realistic-looking wind data on a Gaussian latitude grid.

In [ ]:
# Define grid dimensions
nlat, nlon = 73, 144  # T42 resolution

# Get Gaussian latitudes and weights
lats, wts = gaussian_lats_wts(nlat)
lons = np.linspace(0, 360, nlon, endpoint=False)

# Create longitude and latitude meshgrid
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Create realistic wind patterns
# Zonal wind with jet stream-like structure
u = 20 * np.exp(-((lat_grid - 30)**2) / (2 * 15**2)) + \
    15 * np.exp(-((lat_grid + 30)**2) / (2 * 15**2)) + \
    5 * np.random.randn(nlat, nlon)

# Meridional wind with wave-like patterns
v = 5 * np.sin(3 * np.deg2rad(lon_grid)) * np.cos(np.deg2rad(lat_grid)) + \
    3 * np.random.randn(nlat, nlon)

print(f"Wind data shape: u={u.shape}, v={v.shape}")
print(f"Latitude range: {lats.min():.1f}° to {lats.max():.1f}°")
print(f"Longitude range: {lons.min():.1f}° to {lons.max():.1f}°")

## Initialize VectorWind Object

Create a VectorWind instance for spherical harmonic analysis.

In [ ]:
# Initialize VectorWind with Gaussian grid
vw = VectorWind(u, v, gridtype='gaussian')

print(f"VectorWind initialized:")
print(f"  Grid type: {vw.gridtype}")
print(f"  Latitude points: {vw.nlat}")
print(f"  Longitude points: {vw.nlon}")
print(f"  Earth radius: {vw.s.rsphere/1e6:.2f} x 10^6 m")

## Computing Dynamical Quantities

### Vorticity and Divergence

Calculate the vorticity (circulation) and divergence (spreading) of the wind field.

In [ ]:
# Compute vorticity and divergence
vorticity = vw.vorticity()
divergence = vw.divergence()

print(f"Vorticity statistics:")
print(f"  Min: {vorticity.min():.2e} s⁻¹")
print(f"  Max: {vorticity.max():.2e} s⁻¹")
print(f"  Mean: {vorticity.mean():.2e} s⁻¹")

print(f"\nDivergence statistics:")
print(f"  Min: {divergence.min():.2e} s⁻¹")
print(f"  Max: {divergence.max():.2e} s⁻¹")
print(f"  Mean: {divergence.mean():.2e} s⁻¹")

### Streamfunction and Velocity Potential

Compute the streamfunction (related to rotation) and velocity potential (related to divergence).

In [ ]:
# Compute streamfunction and velocity potential
streamfunction = vw.streamfunction()
velocity_potential = vw.velocitypotential()

# Or compute both at once
psi, chi = vw.sfvp()

print(f"Streamfunction statistics:")
print(f"  Min: {streamfunction.min():.2e} m²/s")
print(f"  Max: {streamfunction.max():.2e} m²/s")
print(f"  Mean: {streamfunction.mean():.2e} m²/s")

print(f"\nVelocity potential statistics:")
print(f"  Min: {velocity_potential.min():.2e} m²/s")
print(f"  Max: {velocity_potential.max():.2e} m²/s")
print(f"  Mean: {velocity_potential.mean():.2e} m²/s")

# Verify that both methods give the same result
assert np.allclose(streamfunction, psi, rtol=1e-10)
assert np.allclose(velocity_potential, chi, rtol=1e-10)
print("\n✓ Verified: sfvp() gives same results as individual functions")

## Helmholtz Decomposition

Decompose the wind field into rotational (non-divergent) and divergent (irrotational) components.

In [ ]:
# Full Helmholtz decomposition
u_rot, v_rot, u_div, v_div = vw.helmholtz()

# Verify the decomposition
u_reconstructed = u_rot + u_div
v_reconstructed = v_rot + v_div

print(f"Helmholtz decomposition verification:")
print(f"  U reconstruction error: {np.abs(u - u_reconstructed).max():.2e}")
print(f"  V reconstruction error: {np.abs(v - v_reconstructed).max():.2e}")

# Calculate energy in each component
total_energy = np.mean(u**2 + v**2)
rotational_energy = np.mean(u_rot**2 + v_rot**2)
divergent_energy = np.mean(u_div**2 + v_div**2)

print(f"\nEnergy decomposition:")
print(f"  Total energy: {total_energy:.3f} m²/s²")
print(f"  Rotational energy: {rotational_energy:.3f} m²/s² ({100*rotational_energy/total_energy:.1f}%)")
print(f"  Divergent energy: {divergent_energy:.3f} m²/s² ({100*divergent_energy/total_energy:.1f}%)")
print(f"  Energy conservation: {rotational_energy + divergent_energy:.3f} m²/s²")

## Alternative Component Access

Get rotational and divergent components separately.

In [ ]:
# Get components separately
u_nondiv, v_nondiv = vw.nondivergentcomponent()
u_irrot, v_irrot = vw.irrotationalcomponent()

# Verify these match the Helmholtz decomposition
assert np.allclose(u_rot, u_nondiv, rtol=1e-10)
assert np.allclose(v_rot, v_nondiv, rtol=1e-10)
assert np.allclose(u_div, u_irrot, rtol=1e-10)
assert np.allclose(v_div, v_irrot, rtol=1e-10)

print("✓ Verified: Individual component methods match Helmholtz decomposition")

## Gradient Operations

Compute gradients of scalar fields.

In [ ]:
# Compute gradient of streamfunction (should give rotational wind)
psi_grad_u, psi_grad_v = vw.gradient(streamfunction)

# The gradient of streamfunction should be related to the rotational wind
# u_rot = -∂ψ/∂y, v_rot = ∂ψ/∂x (with appropriate scaling)
print(f"Gradient computation:")
print(f"  Streamfunction gradient U shape: {psi_grad_u.shape}")
print(f"  Streamfunction gradient V shape: {psi_grad_v.shape}")

# Check the relationship (note: exact match depends on truncation)
correlation_u = np.corrcoef(psi_grad_u.flatten(), (-v_rot).flatten())[0, 1]
correlation_v = np.corrcoef(psi_grad_v.flatten(), u_rot.flatten())[0, 1]

print(f"  Correlation between ∂ψ/∂x and u_rot: {correlation_v:.3f}")
print(f"  Correlation between ∂ψ/∂y and -v_rot: {correlation_u:.3f}")

## Visualization

Create visualizations of the computed fields.

In [ ]:
# Create a comprehensive plot
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Vector Wind Analysis Results', fontsize=16, fontweight='bold')

# Plot 1: Original wind vectors
ax = axes[0, 0]
skip = 4  # Skip every 4th point for clearer visualization
ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip], 
          u[::skip, ::skip], v[::skip, ::skip], 
          scale=300, alpha=0.7)
ax.set_title('Original Wind Field')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.grid(True, alpha=0.3)

# Plot 2: Vorticity
ax = axes[0, 1]
im = ax.contourf(lon_grid, lat_grid, vorticity*1e5, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Relative Vorticity (×10⁻⁵ s⁻¹)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 3: Divergence
ax = axes[0, 2]
im = ax.contourf(lon_grid, lat_grid, divergence*1e5, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Divergence (×10⁻⁵ s⁻¹)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 4: Streamfunction
ax = axes[1, 0]
im = ax.contourf(lon_grid, lat_grid, streamfunction*1e-6, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Streamfunction (×10⁶ m²/s)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 5: Velocity Potential
ax = axes[1, 1]
im = ax.contourf(lon_grid, lat_grid, velocity_potential*1e-6, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Velocity Potential (×10⁶ m²/s)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 6: Rotational vs Divergent Components
ax = axes[1, 2]
ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip], 
          u_rot[::skip, ::skip], v_rot[::skip, ::skip], 
          color='blue', scale=300, alpha=0.7, label='Rotational')
ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip], 
          u_div[::skip, ::skip], v_div[::skip, ::skip], 
          color='red', scale=300, alpha=0.7, label='Divergent')
ax.set_title('Wind Components')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Time Series Analysis

Demonstrate analysis of time-varying wind fields.

In [ ]:
# Create time-varying wind data
nt = 12  # 12 time steps (e.g., months)
time = np.arange(nt)

# Add time dimension to wind data with seasonal variation
u_time = np.zeros((nlat, nlon, nt))
v_time = np.zeros((nlat, nlon, nt))

for t in range(nt):
    # Seasonal modulation
    seasonal_factor = 1 + 0.3 * np.cos(2 * np.pi * t / 12)
    u_time[:, :, t] = u * seasonal_factor + 2 * np.random.randn(nlat, nlon)
    v_time[:, :, t] = v * seasonal_factor + 2 * np.random.randn(nlat, nlon)

# Initialize VectorWind with time series
vw_time = VectorWind(u_time, v_time, gridtype='gaussian')

# Compute vorticity time series
vorticity_time = vw_time.vorticity()

print(f"Time series analysis:")
print(f"  Input wind shape: {u_time.shape}")
print(f"  Vorticity shape: {vorticity_time.shape}")

# Calculate global mean vorticity for each time step
# Use Gaussian weights for proper global averaging
weights_2d = np.broadcast_to(wts[:, np.newaxis], (nlat, nlon))
global_vort = np.array([np.average(vorticity_time[:, :, t], weights=weights_2d) 
                       for t in range(nt)])

# Plot time series
plt.figure(figsize=(10, 6))
plt.plot(time, global_vort * 1e5, 'bo-', linewidth=2, markersize=8)
plt.title('Global Mean Vorticity Time Series', fontsize=14, fontweight='bold')
plt.xlabel('Time Step')
plt.ylabel('Vorticity (×10⁻⁵ s⁻¹)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nGlobal vorticity statistics:")
print(f"  Mean: {global_vort.mean():.2e} s⁻¹")
print(f"  Std: {global_vort.std():.2e} s⁻¹")
print(f"  Range: [{global_vort.min():.2e}, {global_vort.max():.2e}] s⁻¹")

## Error Handling and Validation

Demonstrate the comprehensive error handling in the module.

In [ ]:
# Example 1: Shape mismatch
try:
    vw_error = VectorWind(u[:, :50], v, gridtype='gaussian')
except ValueError as e:
    print(f"✓ Caught shape mismatch error: {e}")

# Example 2: Invalid grid type
try:
    vw_error = VectorWind(u, v, gridtype='invalid')
except ValueError as e:
    print(f"✓ Caught invalid grid type error: {e}")

# Example 3: Data with NaN values
u_nan = u.copy()
u_nan[0, 0] = np.nan
try:
    vw_error = VectorWind(u_nan, v, gridtype='gaussian')
except ValueError as e:
    print(f"✓ Caught NaN values error: {e}")

# Example 4: Insufficient latitude points
try:
    vw_error = VectorWind(u[:2, :], v[:2, :], gridtype='gaussian')
except ValueError as e:
    print(f"✓ Caught insufficient latitude points error: {e}")

print("\n✓ All error handling examples completed successfully!")

## Performance Comparison

Compare performance of different computation modes.

In [ ]:
import time

# Test performance with different legfunc settings
print("Performance comparison:")
print("-" * 40)

# Stored mode (default)
start_time = time.time()
vw_stored = VectorWind(u, v, gridtype='gaussian', legfunc='stored')
vort_stored = vw_stored.vorticity()
time_stored = time.time() - start_time

# Computed mode
start_time = time.time()
vw_computed = VectorWind(u, v, gridtype='gaussian', legfunc='computed')
vort_computed = vw_computed.vorticity()
time_computed = time.time() - start_time

print(f"Stored legfunc:   {time_stored:.4f} seconds")
print(f"Computed legfunc: {time_computed:.4f} seconds")
print(f"Speedup factor:   {time_computed/time_stored:.2f}x")

# Verify results are identical
assert np.allclose(vort_stored, vort_computed, rtol=1e-10)
print("✓ Results are identical regardless of legfunc setting")

## Summary

This notebook demonstrated the key capabilities of the `skyborn.windspharm` module:

1. **Vector Wind Analysis**: Computing vorticity, divergence, streamfunction, and velocity potential
2. **Helmholtz Decomposition**: Separating wind into rotational and divergent components
3. **Time Series Analysis**: Handling time-varying wind fields
4. **Error Handling**: Comprehensive validation and error reporting
5. **Performance Options**: Different computation modes for speed/memory trade-offs

The module provides a modern, well-documented interface for spherical harmonic vector wind analysis, making it easy to analyze atmospheric and oceanic wind patterns with high accuracy and efficiency.